In [165]:
import datetime
from pathlib import Path
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
import warnings
import requests
import holidays
import copy
import json
import re
import os

import output
from impala.dbapi import connect
from impala.util import as_pandas

warnings.filterwarnings(action='ignore')


path = Path.cwd().parent
raw_data = str(path) + '/raw_data/'
data = str(path) + '/data/'
if not os.path.exists(str(path)+'/data/'):
    os.makedirs(str(path)+'/data/')

#### 향후 수정사항
def slicing(df,i) :
    pattern = '.*동'
    reg = re.search(pattern, df['장비명'][i])
    reg = reg.group()
    df['주차장'][i] = reg
    return reg
def change(x):
    if x =='13-1동' or x =='13-2동' or x =='13-3동':
        return '13동'
    elif x == '14-1동' or x == '14-2동':
        return '14동'
    else :
        return x


def parking(x):
    if x =='15동':
        return 15
    elif x == '14동':
        return 14
    elif x == '13동':
        return 13
    elif x == '12동':
        return 12
    elif x == '주차1동,주차2동':
        return 0
    else :
        return x


def confusion(df):
    if df['비율'] <= 0.3:
        return 0
    elif df['비율'] <= 0.5:
        return 1
    elif df['비율'] <= 0.8:
        return 2
    else:
        return

satsun = ['Independence Movement Day','Liberation Day','National Foundation Day','Hangeul Day',"Children's Day"]
kr_holidays = holidays.KR()
def holiday_check(date):
    if date in kr_holidays:
        return 1
    elif date.weekday() == 0:
        if (kr_holidays.get(datetime.datetime.strftime(date-relativedelta(days=1),"%Y-%m-%d")) or kr_holidays.get(datetime.datetime.strftime(date-relativedelta(days=2),"%Y-%m-%d"))) in satsun:
            return 1
        else:
            return 0
    else:
        return 0


def minute_yn(df):
    num = int(60/time_delta)
    for i in range(num):
        if df['분'] <= 10*i + 9:
            return i




def replace_rain(df):
    if df['강수량(mm)'] == '1.0mm미만':
        return 0.5
    elif df['강수량(mm)'] == '30.0~50.0mm':
        return 40
    elif df['강수량(mm)'] == '50.0mm이상':
        return 60
    else:
        return float(str(df['강수량(mm)']).replace('mm', ''))


def replace_snow(df):
    if df['적설(cm)'] == '1cm미만':
        return 0.5
    elif df['적설(cm)'] == '5cm이상':
        return 5
    else:
        return float(str(df['적설(cm)']).replace('cm', ''))


In [166]:
date = output.date
# 예측시작시간
first_pred_time = 7
# 예측종료시간
finish_pred_time = 10
#예측시간간격
time_delta = 10
# conn = connect(host = '172.31.101.61', port = 21050 , user = 'hive', password = 'hive')

#### 데이터 로드
print(datetime.datetime.now().strftime('%Y.%m.%d %Hh%Mm%Ss'), 'Load Data')
darae_in = pd.read_csv(raw_data+'VW_InCarinfo (차량 입차 정보 조회).csv').dropna()
darae_out = pd.read_csv(raw_data+'VW_OutCarinfo (차량 출차 정보 조회).csv')
hanmaek_ticket =  pd.read_csv(raw_data+'일반차량.csv')
hanmanek_normal = pd.read_csv(raw_data+'정기권차량.csv')
parking_info = pd.read_excel(raw_data+'주차장면수.xlsx')


2022.12.23 11h09m35s Load Data


# 날씨데이터 불러오기

In [167]:

# conn = connect(host = '172.31.101.61', port = 21050 , user = 'hive', password = 'hive')
# create_query ='select * from SVC.ext_kma_asos_hourly_info_nopartition'
# # run query on impala
# cur = conn.cursor()
# cur.execute(create_query)
# whether_his = as_pandas(cur)
#
# create_query ='select * from SVC.ext_kma_vilage_fcst_nopartition'
# # run query on impala
# cur = conn.cursor()
# cur.execute(create_query)
# whether_pre = as_pandas(cur)
# cur.close()
#
# # 예보데이터
# df_tmp = whether_pre[whether_pre["category"]=="TMP"][["fcstdate","fcsttime","fcstvalue"]].rename(columns={
#     "fcstdate":"일시","fcsttime":"시간","fcstvalue":"기온(°C)"})
# df_pcp = whether_pre[whether_pre["category"]=="PCP"][["fcstdate","fcsttime","fcstvalue"]].rename(columns={
#     "fcstdate":"일시","fcsttime":"시간","fcstvalue":"강수량(mm)"})
# df_wsd = whether_pre[whether_pre["category"]=="WSD"][["fcstdate","fcsttime","fcstvalue"]].rename(columns={
#     "fcstdate":"일시","fcsttime":"시간","fcstvalue":"풍속(m/s)"})
# df_reh = whether_pre[whether_pre["category"]=="REH"][["fcstdate","fcsttime","fcstvalue"]].rename(columns={
#     "fcstdate":"일시","fcsttime":"시간","fcstvalue":"습도(%)"})
# df_sno = whether_pre[whether_pre["category"]=="SNO"][["fcstdate","fcsttime","fcstvalue"]].rename(columns={
#     "fcstdate":"일시","fcsttime":"시간","fcstvalue":"적설(cm)"})
#
# df_pcp.loc[df_pcp["강수량(mm)"]=="강수없음","강수량(mm)"] = 0
# df_sno.loc[df_sno["적설(cm)"]=="적설없음","적설(cm)"] = 0
#
# final_df = df_tmp.copy()
# final_df = pd.merge(final_df,df_pcp,on=["일시","시간"],how="left")
# final_df = pd.merge(final_df,df_wsd,on=["일시","시간"],how="left")
# final_df = pd.merge(final_df,df_reh,on=["일시","시간"],how="left")
# final_df = pd.merge(final_df,df_sno,on=["일시","시간"],how="left")
#
# for ind in range(len(final_df)):
#     # final_df.loc[ind, "일시"] = datetime.datetime.strptime(final_df.loc[ind,"일시"][:4]+"-"+final_df.loc[ind,"일시"][4:6]+"-"+final_df.loc[ind,"일시"][6:]+" "+final_df.loc[ind,"시간"][:2],"%Y-%m-%d %H")
#     # from datetime import datetime으로 바뀌었음 --> datetime.datetime을 datetime으로 변경
#     final_df.loc[ind, "일시"] = datetime.strptime(final_df.loc[ind,"일시"][:4]+"-"+final_df.loc[ind,"일시"][4:6]+"-"+final_df.loc[ind,"일시"][6:]+" "+final_df.loc[ind,"시간"][:2],"%Y-%m-%d %H")
#
# whether_pre = final_df.drop("시간",axis=1)
# whether_pre['적설(cm)'] = whether_pre.apply(replace_snow,axis=1)
# whether_pre['강수량(mm)'] = whether_pre.apply(replace_rain,axis=1)
# #과거데이터
# whether_his = whether_his[['tm','ta','rn','ws','hm','dsnw']]
# whether_his.replace('',0,inplace=True)
#
# whether_his.columns = whether_pre.columns

whether_his = pd.read_csv('whether_his.csv')
whether_pre = pd.read_csv('whether_pre.csv')
whether_his['일시'] = pd.to_datetime(whether_his['일시'])
whether_pre['일시'] = pd.to_datetime(whether_pre['일시'])
whether_pre

,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),연,월,일,시,분
0,2022-12-21 07:00:00,-1,1.0,2.3,95,0.5,2022,12,21,7,0
1,2022-12-21 08:00:00,0,1.0,2.3,95,0.5,2022,12,21,8,0
2,2022-12-21 09:00:00,1,1.0,2.1,95,0.0,2022,12,21,9,0
3,2022-12-21 10:00:00,2,1.0,2.6,90,0.0,2022,12,21,10,0
4,2022-12-22 07:00:00,-6,0.0,2.5,85,0.0,2022,12,22,7,0
5,2022-12-22 08:00:00,-6,0.0,2.3,85,0.0,2022,12,22,8,0
6,2022-12-22 09:00:00,-6,0.0,3.6,80,0.5,2022,12,22,9,0
7,2022-12-22 10:00:00,-5,0.0,4.1,70,0.5,2022,12,22,10,0
8,2022-12-23 07:00:00,-13,0.0,0.9,90,0.5,2022,12,23,7,0
9,2022-12-23 08:00:00,-12,0.0,0.9,90,0.5,2022,12,23,8,0


# 한맥데이터 전처리

In [168]:
def pretreatment(df):
    df['일시'] = pd.to_datetime(df['FilePath'].apply(lambda x: x[12:26]), format='%Y%m%d%H%M%S')
    df['일시'] = pd.to_datetime(df['일시'])
    df = df[['일시',  'InOutDiv','CarNo']]
    return df

def del_min_sec(x):
    return x.replace(minute=0, second=0)

hanmaek_ticket = pretreatment(hanmaek_ticket)

hanmanek_normal = pretreatment(hanmanek_normal)


In [169]:
hanmaek = pd.concat([hanmaek_ticket,hanmanek_normal])
hanmaek['주차장'] = '주차1동,주차2동'
hanmaek = hanmaek[['CarNo','주차장','InOutDiv','일시']]
hanmaek.columns = ['차량번호','주차장','입출차여부','입출차시각']
hanmaek['입출차여부'] = hanmaek['입출차여부'].apply(lambda x : '입차' if x ==1 else '출차')
hanmaek

,차량번호,주차장,입출차여부,입출차시각
0,367다8120,"주차1동,주차2동",출차,2022-10-01 22:45:59
1,29서8759,"주차1동,주차2동",입차,2022-10-01 22:03:47
2,31도7993,"주차1동,주차2동",출차,2022-10-01 21:57:40
3,70수6806,"주차1동,주차2동",출차,2022-10-01 21:48:00
4,382라2083,"주차1동,주차2동",출차,2022-10-01 21:37:36
...,...,...,...,...
42608,384너2711,"주차1동,주차2동",출차,2020-10-01 16:34:17
42609,384너2711,"주차1동,주차2동",입차,2020-10-01 14:38:20
42610,21어9903,"주차1동,주차2동",출차,2020-10-01 14:06:49
42611,36조5492,"주차1동,주차2동",입차,2020-10-01 08:31:22


# 다래전처리

In [170]:
#### 전처리
print(datetime.datetime.now().strftime('%Y.%m.%d %Hh%Mm%Ss'), 'Preprocessing Data')
# 공백 제거
darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']] = darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip())
darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']] = darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip())



2022.12.23 11h09m36s Preprocessing Data


In [171]:
# 장비명에서 동만 추출하여 주차장에 입력
darae_in['주차장'] = 0
for i in range(len(darae_in)):
    slicing(darae_in,i)
darae_out['주차장'] = 0
for i in range(len(darae_out)):
    slicing(darae_out,i)

# 13-1, 2, 3, 14-1, 2동을 13동, 14동으로 변경
# darae_in['주차장'] = darae_in['주차장'].apply(change)
# darae_out['주차장'] = darae_out['주차장'].apply(change)
test_in = copy.deepcopy(darae_in)
test_in.columns = ['일자','시각','차량번호','주차일련번호','입출차카드번호','장비명','입출차구분','입출차고객구분','출차여부','차량종류','주차장']
test_in['입출차여부'] = '입차'
test_out = copy.deepcopy(darae_out)
test_out.columns = ['일자','시각','차량번호','주차일련번호','입출차카드번호','장비명','입출차구분','입출차고객구분','출차여부','차량종류','주차장']
test_out['입출차여부'] = '출차'

test_in['입출차시각'] = test_in['일자'] + test_in['시각']
test_in['입출차시각'] = pd.to_datetime(test_in['입출차시각'], format= '%Y-%m-%d %H:%M:%S', errors='raise')

test_out['입출차시각'] = test_out['일자'] + test_out['시각']
test_out['입출차시각'] = pd.to_datetime(test_out['입출차시각'], format= '%Y-%m-%d %H:%M:%S', errors='raise')

test_out = test_out[['차량번호','주차장','입출차여부','입출차시각']]
test_in = test_in[['차량번호','주차장','입출차여부','입출차시각']]
darae = pd.concat([test_in, test_out])

# 병합 및 전처리

In [172]:

# 입차, 출차 데이터 합치고 시간으로 정렬
df = pd.concat([hanmaek,darae]).sort_values(by=['차량번호','입출차시각'])
df['주차장'] = df['주차장'].apply(change)
# import datetime ############ import는 맨 위로 빼둠
# 입출차시각 생성
df['시'] = df['입출차시각'].dt.hour
# df.reset_index(inplace = True)
df.reset_index(inplace=True, drop=True) ######### 어차피 밑에서 index 삭제함 --> reset_index할 때 drop
df=df[['차량번호','주차장','입출차여부','입출차시각']]

In [173]:
df

,차량번호,주차장,입출차여부,입출차시각
0,01가0491,"주차1동,주차2동",입차,2021-03-31 14:47:55
1,01가0491,"주차1동,주차2동",출차,2021-03-31 15:35:43
2,01가0491,"주차1동,주차2동",입차,2022-05-01 11:57:55
3,01가0491,"주차1동,주차2동",출차,2022-05-01 12:39:45
4,01가0703,"주차1동,주차2동",입차,2021-04-24 13:52:11
...,...,...,...,...
242383,충북80배1724,13동,출차,2022-10-05 15:29:16
242384,충북86자3167,13동,입차,2022-09-10 16:07:19
242385,충북86자3167,13동,입차,2022-09-28 17:26:49
242386,충북86자3167,13동,출차,2022-09-28 17:26:49


In [110]:
car_arr = np.array(df[['입출차시각','차량번호']])
test_idx_list =[]

# 입차 시간과 출차시간이 같은 경우를 test_idx_list에 추가
for idx in range(len(car_arr)-1):
    # if car_arr[idx][0] == car_arr[idx+1][0] and car_arr[idx][1] == car_arr[idx+1][1] and car_arr[idx][2] == car_arr[idx+1][2]:
    if (car_arr[idx][0] == car_arr[idx+1][0]) and (car_arr[idx][1] == car_arr[idx+1][1]): ###### 가독성을 위해 조건에 괄호 쳐줌
        test_idx_list.append(idx)
        test_idx_list.append(idx+1)
test_idx_list = list(set(test_idx_list))
test_df = df.loc[test_idx_list]
# 입출차 시간이 같은 경우의 출차 삭제
eleminate_index = test_df[test_df['입출차여부']=='출차'].index
df2 = df.drop(eleminate_index).reset_index(drop=True) ############### 위에서 drop해서 index 컬럼 사라짐, 밑에 있는 reset_index를 위에 붙이고 index를 한번에 drop
# df2.drop(['index'],axis = 1 , inplace = True)
# df2.reset_index(inplace=True)
# df2.drop(['index'],axis = 1 , inplace = True)


# 아래 주석의 두 경우만 남김
car_arr = np.array(df2[['차량번호','입출차여부']])
idx_list = []
for idx in range(len(car_arr)):
    if idx == 0:
        idx_list.append(idx)
    else:
        # 차량 번호가 같고 상태가 다른 경우(입차 > 출차 or 출차 > 입차)
        if (car_arr[idx][0] == car_arr[idx-1][0]) and (car_arr[idx][1] != car_arr[idx-1][1]):
            idx_list.append(idx)
        #차량번호가 바뀌는 경우
        elif car_arr[idx][0] != car_arr[idx-1][0]:
            idx_list.append(idx)
process_df = df2.loc[idx_list]

In [111]:
# 연, 월, 일, 시, 분, 초 컬럼 생성
process_df['연'] = process_df['입출차시각'].dt.year
process_df['월'] = process_df['입출차시각'].dt.month
process_df['일'] = process_df['입출차시각'].dt.day
process_df['시'] = process_df['입출차시각'].dt.hour
process_df['분'] = process_df['입출차시각'].dt.minute
process_df['초'] = process_df['입출차시각'].dt.second
# 출근시간 (7시~9시) 사이만 남김
process_df = process_df[(process_df['시']>=first_pred_time) & (process_df['시']<=finish_pred_time-1)]

# parking_info = pd.read_excel(data_path + '/raw_data/주차장면수.xlsx') ### 파일 읽어오는 부분이므로 맨 위로 보냄
# 주차장 면수와 합치고 필요한 컬럼만 남김
# !!!!!!!!!!!!!!!!!! 13-1동옥외, 13-2동옥외 등은 고려하지 않는 건지? (따로 정리해뒀다가 데이터 붙일 때 현업한테 얘기해야할듯)
dataset = pd.merge(process_df, parking_info[['장소','합계']], left_on='주차장', right_on='장소')
dataset = dataset[['입출차시각','연','월','일','시','분','초','입출차여부','주차장','합계']]
dataset = dataset.sort_values(by=['주차장','입출차시각'])
dataset['잔류차량수'] = 0
arr = np.array(dataset[['연','월','일','시','분','입출차여부','주차장','잔류차량수']])

# 잔류차량수 생성
for idx in range(len(arr)):
    if idx == 0:
        # 첫번째 행이 입차이면 1, 출차이면 -1
        if arr[idx][5]=='입차':
            arr[idx][7] = arr[idx][7] + 1
        else:
            arr[idx][7] = arr[idx][7] - 1
    else:
        pre_time = list(arr[idx-1][[0,1,2,6]])
        now_time = list(arr[idx][[0,1,2,6]])
        status =  arr[idx][5]
        # 이전 데이터와 연,월,일 주차장이 같은 경우
        if pre_time == now_time:
            if status == '입차':
                arr[idx][7] = arr[idx-1][7] + 1
            else:
                arr[idx][7] = arr[idx-1][7] - 1
        #이전 데이터와 연,월,일 주차장이 다른 경우 > 0부터 시작함
        else:
            if status == '입차':
                arr[idx][7] = 1
            else:
                arr[idx][7] = -1
# 사용 데이터 정리 및 비율, 혼잡도 산출
dataset[['연','월','일','시','분','입출차여부','주차장','잔류차량수']] = arr
dataset['비율'] = dataset['잔류차량수'] / dataset['합계']
dataset['혼잡도'] = dataset.apply(confusion,axis=1)
dataset['공휴일'] = dataset['입출차시각'].apply(holiday_check)
dataset['요일'] = dataset['입출차시각'].dt.weekday
dataset = dataset[['입출차시각', '연', '월', '일', '시', '분','초','공휴일','요일','주차장','잔류차량수', '비율','혼잡도']]
# 날짜 매칭이 안되는 데이터들 제거
dataset = dataset.dropna()
dataset['주차장'] = dataset['주차장'].apply(parking)
# 불필요한 컬럼 제거
# dataset.drop('입출차시각', axis=1, inplace= True)
# dataset.drop('초', axis=1, inplace= True) #################### 여러 컬럼을 한번에 제거하면 됨. 잔류차량수와 비율 컬럼도 사용하지 않으므로 제거
dataset.drop(['입출차시각', '초', '잔류차량수', '비율'], axis=1, inplace=True)
dataset

,연,월,일,시,분,공휴일,요일,주차장,혼잡도
532,2022,9,1,7,7,0,3,12,0.0
4679,2022,9,1,7,17,0,3,12,0.0
4569,2022,9,1,7,18,0,3,12,0.0
2015,2022,9,1,7,35,0,3,12,0.0
4680,2022,9,1,7,35,0,3,12,0.0
...,...,...,...,...,...,...,...,...,...
6033,2022,10,1,9,12,0,5,0,0.0
19218,2022,10,1,9,15,0,5,0,0.0
16789,2022,10,1,9,40,0,5,0,0.0
21083,2022,10,1,9,56,0,5,0,0.0


# 날씨데이터 붙이기

1. 학습데이터만들기
- 기존이력 업데이트 안 되었을때 >> 그냥 날짜 최신버전에서 - ~~~년
- 과거날씨 업데이트 안되었을때 >> merge하는 과정에서 어차피 포함 안됨
2. 예측데이터 만들기
- 예측 날씨 업데이트 안되었을때 > 과거데이터의 가장 최근 날씨 가져다 붙이기


In [112]:
## 학습데이터셋 생성

# 연, 월, 일, 시 생성
whether_his['연'] = whether_his['일시'].dt.year
whether_his['월'] = whether_his['일시'].dt.month
whether_his['일'] = whether_his['일시'].dt.day
whether_his['시'] = whether_his['일시'].dt.hour
# 위에서 만든 데이터셋에 날씨 데이터 붙여줌
dataset = pd.merge(dataset, whether_his, on=['연','월','일','시'], how='left')

# import holidays ######### import는 맨 위로 뺌
# 휴일 체크


# 날짜 매칭이 안되는 데이터들 제거
dataset = dataset.dropna()
dataset['주차장'] = dataset['주차장'].apply(parking)
# 불필요한 컬럼 제거
# dataset.drop('입출차시각', axis=1, inplace= True)
# dataset.drop('초', axis=1, inplace= True) #################### 여러 컬럼을 한번에 제거하면 됨. 잔류차량수와 비율 컬럼도 사용하지 않으므로 제

# dataset['tmp'] = dataset.apply(tmp, axis =1)
dataset['minute_yn'] = dataset.apply(minute_yn, axis =1) ############ 컬럼명 tmp에서 minute_yn으로 변경
dataframe = dataset.groupby(['연','월','일','시','minute_yn']).count().reset_index()[['연','월','일','시','minute_yn']]
dataframe['trend'] = range(0, len(dataframe))
dataset = pd.merge(dataset, dataframe, on=['연','월','일','시','minute_yn'], how='left')
dataset.drop('minute_yn', axis=1, inplace=True)

# dataset.index = dataset['trend']
# dataset.drop('trend', axis=1, inplace=True)
dataset.set_index('trend', inplace=True) ################# index설정 후 drop 대신 set_index 활용
dataset.reset_index(inplace=True)
dataset.drop('일시',inplace = True,axis=1)


In [113]:
dataset.head()

,trend,연,월,일,시,분,공휴일,요일,주차장,혼잡도,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm)
0,7341,2022,9,1,7,7,0,3,12,0.0,20.2,0.0,0.9,94,0.0
1,7342,2022,9,1,7,17,0,3,12,0.0,20.2,0.0,0.9,94,0.0
2,7342,2022,9,1,7,18,0,3,12,0.0,20.2,0.0,0.9,94,0.0
3,7344,2022,9,1,7,35,0,3,12,0.0,20.2,0.0,0.9,94,0.0
4,7344,2022,9,1,7,35,0,3,12,0.0,20.2,0.0,0.9,94,0.0


## 예측 데이터셋 생성

In [115]:
print(datetime.datetime.now().strftime('%Y.%m.%d %Hh%Mm%Ss'), 'Make Predict Data')

year = int(date[:4])
month = int(date[4:6])
day = int(date[6:])

len_df = (finish_pred_time -first_pred_time) * 60/time_delta +1

time = datetime.datetime(year, month, day,first_pred_time,0)
delta = datetime.timedelta(days=0, minutes=time_delta)
day_delta = datetime.timedelta(days=2, hours = finish_pred_time - first_pred_time)
time_list = [time]


while time < datetime.datetime(year, month, day,first_pred_time,0)+ day_delta:
    time = time + delta
    time_list.append(time)

time_df = pd.DataFrame(time_list)

time_df['연'] = time_df[0].dt.year
time_df['월'] = time_df[0].dt.month
time_df['일'] = time_df[0].dt.day
time_df['시'] = time_df[0].dt.hour
time_df['분'] = time_df[0].dt.minute
time_df2 = time_df[(time_df['시']== finish_pred_time) & (time_df['분']==0)]
time_df = time_df[(time_df['시']>= first_pred_time) & (time_df['시']<= finish_pred_time-1)]
time_df = pd.concat([time_df,time_df2])
time_df = time_df.sort_values(['연','월','일','시','분'])


parking_list = dataset['주차장'].unique()

predict_dataset=  pd.DataFrame()
for parking in parking_list:
    predict_df = time_df
    predict_df['주차장']=parking
    predict_df = predict_df.sort_values(by=['일','시','분'])
    predict_df['공휴일'] = predict_df[0].apply(holiday_check)
    predict_df['요일'] = predict_df[0].dt.weekday
    predict_df['trend'] = range(dataset['trend'].max()+1, dataset['trend'].max() +1+ len(predict_df) )
    predict_dataset = pd.concat([predict_dataset,predict_df])
predict_dataset.drop(0,axis =1,inplace= True)

whether_pre['일시']= pd.to_datetime(whether_pre['일시'])
whether_pre.drop('분',axis =1 ,inplace = True)
whether_pre['연'] = whether_pre['일시'].dt.year
whether_pre['월'] = whether_pre['일시'].dt.month
whether_pre['일'] = whether_pre['일시'].dt.day
whether_pre['시'] = whether_pre['일시'].dt.hour
predict_dataset = pd.merge(predict_dataset,whether_pre , how = 'left', on=['연','월','일','시'] )
predict_dataset.drop('일시',axis =1,inplace= True)
predict_dataset =  predict_dataset[['trend', '연', '월', '일', '시', '분', '공휴일', '요일', '주차장', '기온(°C)',
       '강수량(mm)', '풍속(m/s)', '습도(%)', '적설(cm)']]
predict_dataset

2022.12.22 16h06m06s Make Predict Data


,trend,연,월,일,시,분,공휴일,요일,주차장,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm)
0,7869,2022,12,21,7,0,0,2,12,-1,1.0,2.3,95,0.5
1,7870,2022,12,21,7,10,0,2,12,-1,1.0,2.3,95,0.5
2,7871,2022,12,21,7,20,0,2,12,-1,1.0,2.3,95,0.5
3,7872,2022,12,21,7,30,0,2,12,-1,1.0,2.3,95,0.5
4,7873,2022,12,21,7,40,0,2,12,-1,1.0,2.3,95,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,7921,2022,12,23,9,20,0,4,0,-11,0.0,1.1,85,0.5
281,7922,2022,12,23,9,30,0,4,0,-11,0.0,1.1,85,0.5
282,7923,2022,12,23,9,40,0,4,0,-11,0.0,1.1,85,0.5
283,7924,2022,12,23,9,50,0,4,0,-11,0.0,1.1,85,0.5


# 날짜가 없을때는??

In [123]:
print(datetime.datetime.now().strftime('%Y.%m.%d %Hh%Mm%Ss'), 'Make Predict Data')

year = int(date[:4])
month = int(date[4:6])
day = int(date[6:])

len_df = (finish_pred_time -first_pred_time) * 60/time_delta +1

time = datetime.datetime(year, month, day,first_pred_time,0)
delta = datetime.timedelta(days=0, minutes=time_delta)
day_delta = datetime.timedelta(days=2, hours = finish_pred_time - first_pred_time)
time_list = [time]


while time < datetime.datetime(year, month, day,first_pred_time,0)+ day_delta:
    time = time + delta
    time_list.append(time)

time_df = pd.DataFrame(time_list)

time_df['연'] = time_df[0].dt.year
time_df['월'] = time_df[0].dt.month
time_df['일'] = time_df[0].dt.day
time_df['시'] = time_df[0].dt.hour
time_df['분'] = time_df[0].dt.minute
time_df2 = time_df[(time_df['시']== finish_pred_time) & (time_df['분']==0)]
time_df = time_df[(time_df['시']>= first_pred_time) & (time_df['시']<= finish_pred_time-1)]
time_df = pd.concat([time_df,time_df2])
time_df = time_df.sort_values(['연','월','일','시','분'])


parking_list = dataset['주차장'].unique()

predict_dataset=  pd.DataFrame()
for parking in parking_list:
    predict_df = time_df
    predict_df['주차장']=parking
    predict_df = predict_df.sort_values(by=['일','시','분'])
    predict_df['공휴일'] = predict_df[0].apply(holiday_check)
    predict_df['요일'] = predict_df[0].dt.weekday
    predict_df['trend'] = range(dataset['trend'].max()+1, dataset['trend'].max() +1+ len(predict_df) )
    predict_dataset = pd.concat([predict_dataset,predict_df])
predict_dataset.drop(0,axis =1,inplace= True)

whether_test = pd.read_csv('whether_test.csv')

whether_test['일시']= pd.to_datetime(whether_test['일시'])
whether_test.drop('분',axis =1 ,inplace = True)
whether_test['연'] = whether_test['일시'].dt.year
whether_test['월'] = whether_test['일시'].dt.month
whether_test['일'] = whether_test['일시'].dt.day
whether_test['시'] = whether_test['일시'].dt.hour
predict_dataset = pd.merge(predict_dataset,whether_test , how = 'left', on=['연','월','일','시'] )
predict_dataset.drop('일시',axis =1,inplace= True)
predict_dataset =  predict_dataset[['trend', '연', '월', '일', '시', '분', '공휴일', '요일', '주차장', '기온(°C)',
       '강수량(mm)', '풍속(m/s)', '습도(%)', '적설(cm)']]


# nan값이 있는경우  즉, 예보가 업데이트가 되지 않은 경우 작년 날짜를 가져와서 합친다
if predict_dataset.isnull().sum().max() >=1:
    print(f'예보데이터가 없는 경우가 {predict_dataset.isnull().sum().max()} 개 존재합니다')
    null_data = predict_dataset[predict_dataset['기온(°C)'].isnull()].iloc[:,:9]
    null_data = pd.merge(null_data,whether_his, how = 'left',on =['월','일','시'])
    def yearcompare(df):
        if df['연_x'] - df['연_y'] ==1:
            return 1
        else :
            return 0
    null_data['compare'] = null_data.apply(yearcompare,axis =1)
    null_data =null_data[null_data['compare']==1]
    null_data = null_data[['trend', '연_x', '월', '일', '시', '분', '공휴일', '요일', '주차장', '기온(°C)',
           '강수량(mm)', '풍속(m/s)', '습도(%)', '적설(cm)']]
    null_data.rename(columns={'연_x':'연'},inplace = True)

    not_null = predict_dataset[predict_dataset['기온(°C)'].isnull() == False]
    predict_dataset = pd.concat([null_data,not_null])

2022.12.22 16h15m20s Make Predict Data
예보데이터가 없는 경우가 195 개 존재합니다


In [233]:
# whether_his.to_csv('whether_his.csv',index=False, encoding='utf-8-sig')
# whether_pre.to_csv('whether_pre.csv',index=False, encoding='utf-8-sig')